<a href="https://colab.research.google.com/github/alam2018/ml_prediction_m1/blob/master/arima_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Define if the simulation is for uRLLC or eMBB slice
eMBB = True

# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import pandas as pd

# Importing the training set
if eMBB == True:
  dataset_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/traffic_prediction/data_eMBB/train_data/traffic_simulation.csv', sep=';')
else:
  dataset_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/traffic_prediction/data_URLLC/traffic_simulation.csv', sep=';')

#Insert a new column containing the 'byte' value converted to 'Mb' for training
dataset_train['TotalData_Mb'] = dataset_train.iloc[:, 2].values / 1000000

training_set = dataset_train.iloc[:, 4].values
training_set = training_set.reshape(-1,1)


In [0]:
#Prepare training and test dataset
prediction_size = 60
X = training_set
#train_size = int (round(len(X) * 0.7))
train_size = 3000
test_size = train_size + prediction_size

train = X[0:train_size]
#test = X[train_size:test_size] 
test = X[6000:6060]
predictions = []

#Prepare ARIMA model
from statsmodels.tsa.arima_model import ARIMA

import itertools
p=d=q=range(0,10)
pdq = list(itertools.product(p,d,q))

#print (pdq)

import warnings
warnings.filterwarnings('ignore')
temp_aic = 1000000000000

for param in pdq:
    try:
        model_arima = ARIMA(train,order=param)
        model_arima_fit = model_arima.fit()
        print(param,model_arima_fit.aic)
        if model_arima_fit.aic < temp_aic:
          temp_aic = model_arima_fit.aic
          temp_pdq = param
    except:
        continue

print (temp_pdq, temp_aic)

(6, 1, 8) 27424.27697340341


In [0]:

#p,d,q  p = periods taken for autoregressive model
#d -> Integrated order, difference
# q periods in moving average model
#model_arima = ARIMA(train,order=(6, 1, 8))
model_arima = ARIMA(train,order=temp_pdq)
model_arima_fit = model_arima.fit(disp=0)
#print(model_arima_fit.aic)

predictions= model_arima_fit.forecast(steps=prediction_size)[0]

#print (predictions)

In [0]:
# Visualising the results
plt.plot(test, color = 'red', label = 'Real Traffic (MB)')
plt.plot(predictions, color = 'blue', label = 'Predicted traffic (MB)')
plt.ylim (320,550)
plt.title('Traffic Prediction (ARIMA)')
plt.xlabel('Time Index (s)')
plt.ylabel('Data (MB)')
plt.legend(loc='best', prop={'size': 6})

plt.savefig('arima_valData_data.png',bbox_inches="tight", pad_inches=0, dpi=300)

plt.show()

In [0]:
from sklearn.metrics import mean_squared_error

from math import sqrt

mse = mean_squared_error(test, predictions)

rmse = sqrt(mse)

print('RMSE: %f' % rmse)


#print (predictions.shape)
X_test = np.reshape(test, (np.product(test.shape),))

df = pd.DataFrame(list(zip(X_test, predictions)),
              columns=['Test Packet Size (MB)','Predicted Packet Size (MB)'])

df.to_csv("arima_data.csv", index = None, header=True)
